In [1]:
import math, shutil, warnings
import pandas as pd
from glob import glob
from tabulate import tabulate

In [2]:
# Reading original datasets
paths = glob("data/raw/Vancouver/PM25/*.csv")
paths

['data/raw/Vancouver/PM25/PM25_2018.csv',
 'data/raw/Vancouver/PM25/PM25_2020.csv',
 'data/raw/Vancouver/PM25/PM25_2019.csv',
 'data/raw/Vancouver/PM25/PM25_2017.csv',
 'data/raw/Vancouver/PM25/PM25_2021.csv',
 'data/raw/Vancouver/PM25/PM25_2016.csv']

In [3]:
# Reading datasets
datasets = {}
for path in paths:
    datasets[str(path[24:-4])] = pd.read_csv(path)

datasets_list = sorted(list(datasets.keys()))
datasets_list

['PM25_2016', 'PM25_2017', 'PM25_2018', 'PM25_2019', 'PM25_2020', 'PM25_2021']

In [4]:
# Dataset dimensions
for dataset in datasets_list:
    print(f"{dataset} dim: {datasets[dataset].shape}")

PM25_2016 dim: (529763, 14)
PM25_2017 dim: (606525, 14)
PM25_2018 dim: (557717, 14)
PM25_2019 dim: (566637, 14)
PM25_2020 dim: (564130, 14)
PM25_2021 dim: (869059, 11)


In [5]:
# Summary statistics
for dataset in datasets_list:
    print(f"Summary statistics for {dataset}")
    summary_df = {}
    summary_df["Variable"] = []
    summary_df["Uniques"] = []
    summary_df["Missing"] = []
    summary_df["Min"] = []
    summary_df["Max"] = []
    summary_df["Mean"] = []

    for column in datasets[dataset].columns:
        summary_df["Variable"].append(column)
        uniques = len(pd.unique(datasets[dataset][column]))
        summary_df["Uniques"].append(uniques)
        missing = sum(datasets[dataset][column].isna())
        missing_pct = missing / datasets[dataset].shape[0] * 100
        if missing:
            summary_df["Missing"].append(f"{missing_pct:.2f} %")
        else:
            summary_df["Missing"].append("-")
        if (
            datasets[dataset][column].values.dtype in ["float64", "int64"]
            and "ID" not in column
        ):
            summary_df["Min"].append(round(datasets[dataset][column].min(), 3))
            summary_df["Max"].append(round(datasets[dataset][column].max(), 3))
            summary_df["Mean"].append(round(datasets[dataset][column].mean(), 3))
        else:
            summary_df["Min"].append("-")
            summary_df["Max"].append("-")
            summary_df["Mean"].append("-")

    print(tabulate(pd.DataFrame(summary_df), headers="keys", tablefmt="psql") + "\n")

Summary statistics for PM25_2016
+----+-------------------+-----------+-----------+--------+---------+--------+
|    | Variable          |   Uniques | Missing   | Min    | Max     | Mean   |
|----+-------------------+-----------+-----------+--------+---------+--------|
|  0 | DATE_PST          |      8784 | -         | -      | -       | -      |
|  1 | DATE              |       366 | -         | -      | -       | -      |
|  2 | TIME              |        24 | -         | -      | -       | -      |
|  3 | STATION_NAME      |        63 | -         | -      | -       | -      |
|  4 | STATION_NAME_FULL |        63 | -         | -      | -       | -      |
|  5 | EMS_ID            |        63 | -         | -      | -       | -      |
|  6 | NAPS_ID           |        39 | -         | -      | -       | -      |
|  7 | RAW_VALUE         |    298200 | 6.06 %    | -0.137 | 605.228 | 5.508  |
|  8 | ROUNDED_VALUE     |      1020 | 6.06 %    | -0.1   | 605.2   | 5.508  |
|  9 | UNIT        

In [6]:
# Unique stations
stations = []
for dataset in datasets_list:
    for station in pd.unique(datasets[dataset]["STATION_NAME"]):
        if station not in stations:
            stations.append(station)

print(f"Total unique stations: {len(stations)}")

Total unique stations: 87


In [7]:
# Unique regions
regions = []
for dataset in datasets_list:
    try:
        for region in pd.unique(datasets[dataset]["REGION"]):
            if region not in regions:
                regions.append(region)
    except KeyError:
        pass

print(f"Total unique regions: {len(regions)}")

Total unique regions: 7


In [8]:
# Unique stations per region
stations_per_region = {}
print("Unique stations per region")
for dataset in datasets_list:
    try:
        stations_per_region[dataset[-4:]] = (
            datasets[dataset].groupby("REGION")["STATION_NAME"].nunique()
        )
    except KeyError:
        pass

print(tabulate(pd.concat(stations_per_region, axis=1), headers="keys", tablefmt="psql"))

Unique stations per region
+------------------------+--------+--------+--------+--------+--------+
| REGION                 |   2016 |   2017 |   2018 |   2019 |   2020 |
|------------------------+--------+--------+--------+--------+--------|
| 01 - Vancouver Island  |     11 |     12 |     13 |     11 |     11 |
| 02 - Lower Mainland    |     27 |     27 |     27 |     25 |     25 |
| 03 - Southern Interior |      5 |      5 |      5 |      7 |      3 |
| 04 - Kootenay          |      3 |      4 |      4 |      4 |      4 |
| 05 - Cariboo           |      2 |      2 |      2 |      2 |      3 |
| 06 - Skeena            |     10 |     10 |      8 |      9 |      9 |
| 07 - Omineca-Peace     |      5 |     11 |      9 |      8 |      9 |
+------------------------+--------+--------+--------+--------+--------+


### Subsetting study data
Vancouver Island stations to study:

* Vancouver Clark Drive
* Vancouver International Airport
* Vancouver North Vancouver Mahon Park
* Vancouver North Vancouver Second Narrows

In [9]:
stations = [
    "Vancouver Clark Drive",
    "Vancouver International Airport #2",
    "North Vancouver Mahon Park",
    "North Vancouver Second Narrows",
]

In [10]:
not_founds = []
for dataset in datasets_list:
    for station in stations:
        if station not in datasets[dataset]["STATION_NAME"].unique():
            print(f"{station} station not found in {dataset}")
            not_founds.append(station)
if len(not_founds) == 0:
    print(f"All istations found for datasets: {', '.join(datasets_list)}.")

All istations found for datasets: PM25_2016, PM25_2017, PM25_2018, PM25_2019, PM25_2020, PM25_2021.


In [11]:
# Asserting timeseries integrity and parallelism
columns = ["DATE_PST", "RAW_VALUE"]
starts, ends = [], []
for dataset in datasets_list:
    for station in stations:
        subset = datasets[dataset][columns][
            datasets[dataset]["STATION_NAME"] == station
        ]
        start = min(subset["DATE_PST"])
        end = max(subset["DATE_PST"])
        starts.append(start)
        ends.append(end)
        # Assert time series integrity
        assert (
            len(pd.date_range(start=start, end=end, freq="H")) == subset.shape[0]
        ), f"{dataset}'s date range is incomplete or truncated."

# Appending years and concatenating stations to a master dataframe
master_df = {}
master_df["DATE_PST"] = pd.date_range(start=min(starts), end=max(ends), freq="H")
for station in stations:
    first_df = 1
    for dataset in datasets_list:
        subset = datasets[dataset][columns][
            datasets[dataset]["STATION_NAME"] == station
        ]
        if first_df:
            pm25_complete = subset["RAW_VALUE"]
            first_df = 0
        else:
            pm25_complete = pd.concat(
                [pm25_complete, subset["RAW_VALUE"]], ignore_index=True
            )
    print(f"{station}: {len(pm25_complete)}")
    master_df[station.replace(" ", "_") + "_PM25"] = pm25_complete

master_df = pd.DataFrame(master_df)
master_df.head()

Vancouver Clark Drive: 57014
Vancouver International Airport #2: 57014
North Vancouver Mahon Park: 57014
North Vancouver Second Narrows: 57014


,DATE_PST,Vancouver_Clark_Drive_PM25,Vancouver_International_Airport_#2_PM25,North_Vancouver_Mahon_Park_PM25,North_Vancouver_Second_Narrows_PM25
0,2016-01-01 01:00:00,30.557,22.71692,22.798870,NaN
1,2016-01-01 02:00:00,34.661,21.95020,22.909090,6.341111
2,2016-01-01 03:00:00,35.419,21.25953,19.857730,5.340278
3,2016-01-01 04:00:00,24.335,19.93333,13.575990,3.907917
4,2016-01-01 05:00:00,29.336,19.07922,7.079589,7.235889


In [12]:
# Screening for negative values
for station in stations:
    station_col = station.replace(" ", "_") + "_PM25"
    negatives = master_df[station_col][master_df[station_col] < 0]
    if len(negatives):
        print(f"{len(negatives)} negative values found for {station}:")
        display(negatives)
    else:
        print(f"No negative values for {station}")

7 negative values found for Vancouver Clark Drive:


19874   -0.028597
51793   -0.320931
51794   -0.604097
51795   -0.634181
51796   -0.438903
51800   -0.091346
54003   -0.030625
Name: Vancouver_Clark_Drive_PM25, dtype: float64

No negative values for Vancouver International Airport #2
No negative values for North Vancouver Mahon Park
No negative values for North Vancouver Second Narrows


In [13]:
# Replacing negative values with NaN (missing)
master_df["Vancouver_Clark_Drive_PM25"][
    master_df["Vancouver_Clark_Drive_PM25"] < 0
] = math.nan

/tmp/ipykernel_11187/2839988640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df["Vancouver_Clark_Drive_PM25"][master_df["Vancouver_Clark_Drive_PM25"] < 0] = math.nan


In [14]:
# Screening for negative values
for station in stations:
    station_col = station.replace(" ", "_") + "_PM25"
    negatives = master_df[station_col][master_df[station_col] < 0]
    if len(negatives):
        print(f"{len(negatives)} negative values found for {station}:")
        display(negatives)
    else:
        print(f"No negative values for {station}")

No negative values for Vancouver Clark Drive
No negative values for Vancouver International Airport #2
No negative values for North Vancouver Mahon Park
No negative values for North Vancouver Second Narrows


In [15]:
master_df.to_csv("data/2016_2021_master_df.csv", index=False)